In [1]:
include("./trajopt/utils.jl")
include("./trajopt/dynamics.jl")
include("./funlopt/funl_dynamics.jl")
include("./funlopt/funl_utils.jl")
include("./funlopt/funl_constraint.jl")
include("./trajopt/scaling.jl")
include("./funlopt/funl_synthesis.jl")
using BenchmarkTools
using SparseArrays

In [2]:
# load nominal trajectory
using JLD2, FileIO
@load "./data/quadstar_N15_traj" my_dict
xnom = my_dict["x"]
unom = my_dict["u"]
tnom = my_dict["t"];
N = size(xnom,2) - 1
dtnom = zeros(N)
for i in 1:N
    dtnom[i] = tnom[i+1]-tnom[i]
end

In [3]:
dynamics = QuadrotorDynamics()
ix = dynamics.ix
iu = dynamics.iu
decay_rate = 0.1
DLMI = NonlinearDLMI(decay_rate,ix,iu,dynamics.Cv,dynamics.Dvu)
is = DLMI.is

1

In [4]:
iϕ = dynamics.iϕ
iv = dynamics.iv
@assert size(xnom,2) - 1 == N

In [23]:
idx = 4
A,B = diff(dynamics,xnom[:,idx],unom[:,idx])
Q = rand(ix,ix) .- 0.5

12×12 Matrix{Float64}:
 -0.27113    -0.446484   -0.382454   …  -0.103019   -0.401408   -0.160677
 -0.106919   -0.108262    0.374015      -0.295855    0.210358    0.131026
  0.355285    0.419292   -0.442549       0.260734   -0.251373   -0.100366
 -0.260717   -0.424403    0.170277       0.171242   -0.0301056  -0.0422975
  0.220013   -0.144142    0.137966       0.122867   -0.0758791   0.0924106
  0.317235   -0.0731912  -0.414202   …   0.260673    0.420672   -0.216485
 -0.274068    0.328883   -0.232008      -0.420197    0.478094    0.0440268
 -0.390506   -0.424889   -0.311559       0.162177   -0.41576    -0.388437
 -0.0240105   0.10502     0.0787559      0.221696   -0.268655   -0.101669
  0.491894    0.268533   -0.4385         0.0982411  -0.0232385  -0.432771
  0.459118   -0.234355    0.289864   …   0.280951   -0.341713   -0.0823265
 -0.326525    0.0273339  -0.39184       -0.187617   -0.378502    0.349611

In [6]:
Imat = sparse(Matrix(1.0I,ix,ix))

12×12 SparseMatrixCSC{Float64, Int64} with 12 stored entries:
 1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0

In [7]:
right = A + 0.5*DLMI.alpha*Imat
# Aq = kron(Imat,right) + kron(right,Imat) * model.Cn

12×12 Matrix{Float64}:
 0.05  0.0   0.0   1.0   0.0   0.0   …   0.0        0.0         0.0
 0.0   0.05  0.0   0.0   1.0   0.0       0.0        0.0         0.0
 0.0   0.0   0.05  0.0   0.0   1.0       0.0        0.0         0.0
 0.0   0.0   0.0   0.05  0.0   0.0       0.0        0.0         0.0
 0.0   0.0   0.0   0.0   0.05  0.0       0.0        0.0         0.0
 0.0   0.0   0.0   0.0   0.0   0.05  …   0.0        0.0         0.0
 0.0   0.0   0.0   0.0   0.0   0.0       1.0       -0.1214     -0.338726
 0.0   0.0   0.0   0.0   0.0   0.0       0.0        0.941366   -0.337388
 0.0   0.0   0.0   0.0   0.0   0.0       0.0        0.358565    1.00045
 0.0   0.0   0.0   0.0   0.0   0.0       0.05      -0.264665    0.525795
 0.0   0.0   0.0   0.0   0.0   0.0   …   0.247954   0.05       -0.064055
 0.0   0.0   0.0   0.0   0.0   0.0      -0.112553  -0.0146359   0.05

In [9]:
using BlockDiagonals

function create_block_diagonal(right::Matrix, n::Int)
    blocks = [right for _ in 1:n]
    return BlockDiagonal(blocks)
end
function optimized_kron_identity(right::Matrix{Float64})
    n = size(right, 1)
    m = size(right, 2)
    result = zeros(Float64, n*m, n*m)

    for i in 1:n
        row_index = (i-1)*n + 1
        col_index = (i-1)*m + 1
        result[row_index:row_index+n-1, col_index:col_index+m-1] = right
    end

    return result
end
function optimized_kron_identity_sparse(right::Matrix{T}) where T
    n = size(right, 1)
    m = size(right, 2)
    result = spzeros(T, n*m, n*m)

    for i in 1:n
        row_index = (i-1)*n + 1
        col_index = (i-1)*m + 1
        result[row_index:row_index+n-1, col_index:col_index+m-1] = right
    end

    return result
end

optimized_kron_identity_sparse (generic function with 1 method)

In [10]:
@benchmark kron(Imat,$right)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.746 μs …  1.529 ms  ┊ GC (min … max):  0.00% … 99.57%
 Time  (median):     2.317 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   3.183 μs ± 27.586 μs  ┊ GC (mean ± σ):  27.27% ±  3.57%

    ▁           ▆▆█▃▂ ▂                                       
  ▂▇█▆▇▆▄▂▂▂▂▃▄▇███████▇▆▄▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  1.75 μs        Histogram: frequency by time         3.8 μs <

 Memory estimate: 9.95 KiB, allocs estimate: 9.

In [11]:
@benchmark optimized_kron_identity_sparse($right)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  15.667 μs …   5.818 ms  ┊ GC (min … max):  0.00% … 99.04%
 Time  (median):     19.458 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   24.025 μs ± 128.258 μs  ┊ GC (mean ± σ):  17.29% ±  3.27%

       ▃▁      ▁▃▅▇█▄▃                                          
  ▂▃▅▆████▇▅▆▇▇████████▇▇▆▆▆▆▆▇▆▅▅▄▄▄▃▃▃▃▂▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂ ▄
  15.7 μs         Histogram: frequency by time           29 μs <

 Memory estimate: 87.34 KiB, allocs estimate: 105.

In [12]:
@benchmark optimized_kron_identity_sparse($right)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  15.625 μs …   4.352 ms  ┊ GC (min … max):  0.00% … 98.77%
 Time  (median):     19.459 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   23.861 μs ± 120.678 μs  ┊ GC (mean ± σ):  16.57% ±  3.27%

      ▃▁     ▁▄▅█▄▂                                             
  ▂▃▇████▆▆▆████████▇▆▇▆▇▆▆▅▅▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂ ▄
  15.6 μs         Histogram: frequency by time         31.8 μs <

 Memory estimate: 87.34 KiB, allocs estimate: 105.

In [34]:
@benchmark create_block_diagonal($right, ix)

BenchmarkTools.Trial: 10000 samples with 984 evaluations.
 Range (min … max):  58.520 ns …   4.177 μs  ┊ GC (min … max): 0.00% … 97.61%
 Time  (median):     65.718 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   72.621 ns ± 160.589 ns  ┊ GC (mean ± σ):  9.05% ±  4.02%

  █▆▂  ▃▂▆▂ ▁▂▄▃▅▄▄▄▄▅▄▄▃▃▃▃▃▂▃▂▂▁▁▁ ▁                         ▂
  ██████████████████████████████████████▇▇▇▇▇▆▆▆▆▆▅▅▆▅▅▄▅▆▄▅▃▅ █
  58.5 ns       Histogram: log(frequency) by time      91.5 ns <

 Memory estimate: 144 bytes, allocs estimate: 1.

In [14]:
kron_  = kron(Imat,right)
_kron = kron(right,Imat);

In [15]:
Cn = sparse(DLMI.Cn);

In [16]:
@benchmark kron(right,Imat) * Cn

BenchmarkTools.Trial: 10000 samples with 6 evaluations.
 Range (min … max):  5.181 μs … 883.132 μs  ┊ GC (min … max):  0.00% … 98.79%
 Time  (median):     6.486 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   7.839 μs ±  31.486 μs  ┊ GC (mean ± σ):  16.40% ±  4.06%

     ▁         ▇█▄▂▁▂                                          
  ▁▄▇██▄▃▂▂▃▃▄▇███████▇▆▅▄▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  5.18 μs         Histogram: frequency by time        10.2 μs <

 Memory estimate: 22.17 KiB, allocs estimate: 14.

In [17]:
@benchmark Cn * kron_

BenchmarkTools.Trial: 10000 samples with 8 evaluations.
 Range (min … max):  3.505 μs … 671.260 μs  ┊ GC (min … max):  0.00% … 98.90%
 Time  (median):     4.177 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   4.966 μs ±  21.503 μs  ┊ GC (mean ± σ):  14.91% ±  3.42%

     ▁         █▆█▃▁                                           
  ▂▄▇█▇▃▂▂▂▃▄▆██████▆▇▆▆▅▅▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  3.51 μs         Histogram: frequency by time         6.1 μs <

 Memory estimate: 12.22 KiB, allocs estimate: 5.

In [18]:
@benchmark kron(Imat,Imat)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.038 μs … 415.000 μs  ┊ GC (min … max):  0.00% … 99.06%
 Time  (median):     1.121 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   1.374 μs ±   8.121 μs  ┊ GC (mean ± σ):  11.73% ±  1.98%

  ▂▆█▇▇▅▁ ▁▃▃▂▁   ▃▅▅▄▄▃▂▁▃▄▄▃▃▂▂▁▁▁▁ ▁▁▁ ▁▁▁ ▁▁              ▂
  ███████▆██████▇████████████████████████████████▇███▇▇▇▆▅▆▅▅ █
  1.04 μs      Histogram: log(frequency) by time      1.71 μs <

 Memory estimate: 3.67 KiB, allocs estimate: 6.

In [19]:
@benchmark Matrix(1.0I,ix*ix,ix*ix)

BenchmarkTools.Trial: 10000 samples with 8 evaluations.
 Range (min … max):   1.812 μs …  1.596 ms  ┊ GC (min … max):  0.00% … 99.32%
 Time  (median):     14.755 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   22.553 μs ± 72.875 μs  ┊ GC (mean ± σ):  38.21% ± 12.41%

   █                                                           
  ▄█▅▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂ ▂
  1.81 μs         Histogram: frequency by time         432 μs <

 Memory estimate: 162.05 KiB, allocs estimate: 2.

In [30]:
kron_ = kron(Q',B)
_kron = kron(B,Q')
Cm = sparse(DLMI.Cm);

In [31]:
sum(abs.(kron(B,Q')*Cm - Cn*kron_))

0.0

In [32]:
@benchmark kron(B,Q')*Cm

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   7.583 μs …   5.098 ms  ┊ GC (min … max):  0.00% … 99.40%
 Time  (median):     15.334 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   20.679 μs ± 156.967 μs  ┊ GC (mean ± σ):  26.99% ±  3.57%

                          █▁   ▁                                
  ▃▇▃▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▄██▅▅██▆▄▄▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂ ▃
  7.58 μs         Histogram: frequency by time         24.2 μs <

 Memory estimate: 108.11 KiB, allocs estimate: 5.

In [33]:
@benchmark Cn*kron_

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  12.292 μs …   5.949 ms  ┊ GC (min … max):  0.00% … 99.22%
 Time  (median):     16.375 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   19.802 μs ± 130.486 μs  ┊ GC (mean ± σ):  15.97% ±  2.43%

                   █▁                                           
  ▅█▃▂▂▂▂▂▂▁▂▂▂▂▂▂▂██▇▇▄▃▃▃▃▃▄▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂ ▃
  12.3 μs         Histogram: frequency by time         25.6 μs <

 Memory estimate: 54.05 KiB, allocs estimate: 2.